# Predicting Flight Prices

#### Jake Fox

#### 4/20/2023

This dataset was downloaded from Kaggle. The user from Kaggle received this data through a website called “Ease My Trip” that helps customers plan their vacations and getaways.

In this project, I will be creating a linear regression model to predict flight price based off flight data from a market of airlines based in India.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

## I'm going to read in two datasets that contain flight information for business class and economy class tickets:

In [2]:
business = pd.read_csv(r"D:\business.csv")
economy = pd.read_csv(r"D:\economy.csv")

In [3]:
business.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612"
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612"
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220"
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450"
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690"


In [4]:
economy.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"


## First thing I noticed was the values in column "stop". Let's fix this:

In [5]:
business['stop'].value_counts()

1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t                   81489
non-stop                                                                      8102
2+-stop                                                                       1083
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IDR\n\t\t\t\t\t\t\t\t\t\t\t\t              810
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IXU\n\t\t\t\t\t\t\t\t\t\t\t\t              776
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia PAT\n\t\t\t\t\t\t\t\t\t\t\t\t              257
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Patna\n\t\t\t\t\t\t\t\t\t\t\t\t            242
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia BBI\n\t\t\t\t\t\t\t\t\t\t\t\t              152
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia STV\n\t\t\t\t\t\t\t\t\t\t\t\t               93
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IXE\n\t\t\t\t\t\t\t\t\t\t\t\t               86
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Bhubaneswar\n\t\t\t\t\t\t\t\t\t\t\t\t       75
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Hyderabad\n\t\t\t\t\t\t\t\t\t\t\t\t         71
1-st

In [6]:
business['stop'] = business['stop'].str.slice(0, 8)

business['stop'].value_counts()

1-stop\n\t    84302
non-stop       8102
2+-stop        1083
Name: stop, dtype: int64

In [7]:
business['stop'].replace({'1-stop\n\t': '1-stop'}, inplace = True)

business['stop'].replace({'2+-stop': '2-stops'}, inplace = True)

In [8]:
business['stop'].value_counts()

1-stop      84302
non-stop     8102
2-stops      1083
Name: stop, dtype: int64

In [9]:
business.head(10)

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612"
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612"
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop,20:45,Mumbai,"42,220"
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop,23:55,Mumbai,"44,450"
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop,23:55,Mumbai,"46,690"
5,11-02-2022,Vistara,UK,985,19:50,Delhi,02h 10m,non-stop,22:00,Mumbai,"50,264"
6,11-02-2022,Air India,AI,479,21:15,Delhi,17h 45m,1-stop,15:00,Mumbai,"50,669"
7,11-02-2022,Air India,AI,473,18:40,Delhi,22h 45m,1-stop,17:25,Mumbai,"51,059"
8,11-02-2022,Vistara,UK,871,20:35,Delhi,17h 55m,1-stop,14:30,Mumbai,"51,731"
9,11-02-2022,Vistara,UK,977,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"53,288"


## Now that I've changed the data in the business dataframe, I'll do the same for the economy data:

In [10]:
economy['stop'] = economy['stop'].str.slice(0, 8)

economy['stop'].value_counts()

1-stop\n\t    166627
non-stop       27942
2+-stop        12205
Name: stop, dtype: int64

In [11]:
economy['stop'].replace({'1-stop\n\t': '1-stop'}, inplace = True)

economy['stop'].replace({'2+-stop': '2-stops'}, inplace = True)

In [12]:
economy['stop'].value_counts()

1-stop      166627
non-stop     27942
2-stops      12205
Name: stop, dtype: int64

In [13]:
economy.head(10)

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"
5,11-02-2022,Vistara,UK,945,11:40,Delhi,02h 20m,non-stop,14:00,Mumbai,"5,955"
6,11-02-2022,Vistara,UK,927,09:30,Delhi,02h 05m,non-stop,11:35,Mumbai,"6,060"
7,11-02-2022,Vistara,UK,951,14:20,Delhi,02h 10m,non-stop,16:30,Mumbai,"6,060"
8,11-02-2022,GO FIRST,G8,334,08:00,Delhi,02h 10m,non-stop,10:10,Mumbai,"5,954"
9,11-02-2022,GO FIRST,G8,336,14:20,Delhi,02h 15m,non-stop,16:35,Mumbai,"5,954"


## Now I'll check for null values and duplicates:

In [14]:
business.isnull().sum()

date          0
airline       0
ch_code       0
num_code      0
dep_time      0
from          0
time_taken    0
stop          0
arr_time      0
to            0
price         0
dtype: int64

In [15]:
business_dupe = business.duplicated().sum()
business_dupe

0

In [16]:
economy.isnull().sum()

date          0
airline       0
ch_code       0
num_code      0
dep_time      0
from          0
time_taken    0
stop          0
arr_time      0
to            0
price         0
dtype: int64

In [17]:
economy_dupe = economy.duplicated().sum()
economy_dupe

2

## I can see there are duplicate rows in the economy table. I'll take a look at them and then remove them:

In [18]:
economy_dupe = economy.duplicated(keep=False).sum()

if economy_dupe > 0:
    print('Duplicated rows:')
    print(economy[economy.duplicated(keep=False)])
else:
    print('No duplicated rows')

Duplicated rows:
            date    airline ch_code  num_code dep_time   from time_taken  \
516   14-02-2022  Air India      AI       807    17:20  Delhi    15h 15m   
563   14-02-2022  Air India      AI       807    17:20  Delhi    15h 15m   
6080  13-03-2022  Air India      AI       475    13:00  Delhi    24h 35m   
6181  13-03-2022  Air India      AI       475    13:00  Delhi    24h 35m   

        stop arr_time      to   price  
516   1-stop    08:35  Mumbai  12,150  
563   1-stop    08:35  Mumbai  12,150  
6080  1-stop    13:35  Mumbai   4,780  
6181  1-stop    13:35  Mumbai   4,780  


In [19]:
economy = economy.drop_duplicates()

In [20]:
economy_dupe = economy.duplicated(keep=False).sum()

if economy_dupe > 0:
    print('Duplicated rows:')
    print(economy[economy.duplicated(keep=False)])
else:
    print('No duplicated rows')

No duplicated rows


## Before I combine these dataframes, I will add a column to each called "class" that will say either business or economy:

In [21]:
economy['class'] = "economy"

economy.head(10)

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,class
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953",economy
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953",economy
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956",economy
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955",economy
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955",economy
5,11-02-2022,Vistara,UK,945,11:40,Delhi,02h 20m,non-stop,14:00,Mumbai,"5,955",economy
6,11-02-2022,Vistara,UK,927,09:30,Delhi,02h 05m,non-stop,11:35,Mumbai,"6,060",economy
7,11-02-2022,Vistara,UK,951,14:20,Delhi,02h 10m,non-stop,16:30,Mumbai,"6,060",economy
8,11-02-2022,GO FIRST,G8,334,08:00,Delhi,02h 10m,non-stop,10:10,Mumbai,"5,954",economy
9,11-02-2022,GO FIRST,G8,336,14:20,Delhi,02h 15m,non-stop,16:35,Mumbai,"5,954",economy


In [22]:
business['class'] = "business"

business.head(10)

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,class
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612",business
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612",business
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop,20:45,Mumbai,"42,220",business
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop,23:55,Mumbai,"44,450",business
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop,23:55,Mumbai,"46,690",business
5,11-02-2022,Vistara,UK,985,19:50,Delhi,02h 10m,non-stop,22:00,Mumbai,"50,264",business
6,11-02-2022,Air India,AI,479,21:15,Delhi,17h 45m,1-stop,15:00,Mumbai,"50,669",business
7,11-02-2022,Air India,AI,473,18:40,Delhi,22h 45m,1-stop,17:25,Mumbai,"51,059",business
8,11-02-2022,Vistara,UK,871,20:35,Delhi,17h 55m,1-stop,14:30,Mumbai,"51,731",business
9,11-02-2022,Vistara,UK,977,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"53,288",business


## Combining dataframes and checking the shapes:

In [23]:
df = pd.concat([business, economy])

In [24]:
print(business.shape)
print(economy.shape)
print(df.shape)

(93487, 12)
(206772, 12)
(300259, 12)


In [25]:
93487 + 206772

300259

In [26]:
df.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,class
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612",business
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612",business
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop,20:45,Mumbai,"42,220",business
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop,23:55,Mumbai,"44,450",business
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop,23:55,Mumbai,"46,690",business


In [27]:
df.dtypes

date          object
airline       object
ch_code       object
num_code       int64
dep_time      object
from          object
time_taken    object
stop          object
arr_time      object
to            object
price         object
class         object
dtype: object

## I want price to be type int, not an object
## I need to remove the commas from the values first:

In [28]:
df['price'] = df['price'].str.replace(',', '')

In [29]:
df['price'] = df['price'].astype('int')

In [30]:
df.dtypes

date          object
airline       object
ch_code       object
num_code       int64
dep_time      object
from          object
time_taken    object
stop          object
arr_time      object
to            object
price          int32
class         object
dtype: object

## Let's convert time_taken to an integer and represent it in minutes.

In [31]:
time_taken = df['time_taken'].value_counts()

In [32]:
time_taken

02h 10m    4242
02h 15m    4036
02h 45m    2879
02h 05m    2755
02h 50m    2323
           ... 
38h 45m       1
1.01h m       1
1.02h m       1
38h 30m       1
41h 30m       1
Name: time_taken, Length: 483, dtype: int64

### I need to remove columns that don't match the rest of the values' syntax first. I can see there are 2 values here (1.01h m, 1.02h m) that don't match the rest.

### There is one more value (1.03h m) that you cannot see because the "max rows" option has been reset, but there are two rows that contain that value.

### They will be removed as well, so 4 rows removed total here:

In [33]:
df.shape

(300259, 12)

In [34]:
values_to_remove = ['1.01h m', '1.02h m', '1.03h m']

df = df[~df['time_taken'].isin(values_to_remove)]

In [35]:
df.shape

(300255, 12)

## Now let's actually make it type int:

### I'll first create a function that will convert the time_taken into total minutes. Then, change to type int:

In [36]:
df[['time_taken']]

,time_taken
0,02h 00m
1,02h 15m
2,24h 45m
3,26h 30m
4,06h 40m
...,...
206769,13h 50m
206770,13h 50m
206771,20h 35m
206772,23h 20m


In [37]:
def convert_time_to_minutes(time_str):
    hours, minutes = time_str.split('h ')
    minutes = int(hours) * 60 + int(minutes[:-1])
    return minutes

df['time_taken'] = df['time_taken'].apply(convert_time_to_minutes).astype(int)

In [38]:
df[['time_taken']]

,time_taken
0,120
1,135
2,1485
3,1590
4,400
...,...
206769,830
206770,830
206771,1235
206772,1400


## I'll rename 'time_taken' to address the change to total minutes:

In [39]:
df = df.rename(columns={'time_taken': 'time_taken_mins'})

In [40]:
df.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken_mins,stop,arr_time,to,price,class
0,11-02-2022,Air India,AI,868,18:00,Delhi,120,non-stop,20:00,Mumbai,25612,business
1,11-02-2022,Air India,AI,624,19:00,Delhi,135,non-stop,21:15,Mumbai,25612,business
2,11-02-2022,Air India,AI,531,20:00,Delhi,1485,1-stop,20:45,Mumbai,42220,business
3,11-02-2022,Air India,AI,839,21:25,Delhi,1590,1-stop,23:55,Mumbai,44450,business
4,11-02-2022,Air India,AI,544,17:15,Delhi,400,1-stop,23:55,Mumbai,46690,business


In [41]:
df['airline'].value_counts()

Vistara      127859
Air India     80892
Indigo        43120
GO FIRST      23173
AirAsia       16098
SpiceJet       9011
StarAir          61
Trujet           41
Name: airline, dtype: int64

In [42]:
df['ch_code'].value_counts()

UK    127859
AI     80892
6E     43120
G8     23173
I5     16098
SG      9011
S5        61
2T        41
Name: ch_code, dtype: int64

## I'm going to drop ch_code. It is redundant:

In [43]:
df = df.drop('ch_code', axis=1)

df.head(10)

,date,airline,num_code,dep_time,from,time_taken_mins,stop,arr_time,to,price,class
0,11-02-2022,Air India,868,18:00,Delhi,120,non-stop,20:00,Mumbai,25612,business
1,11-02-2022,Air India,624,19:00,Delhi,135,non-stop,21:15,Mumbai,25612,business
2,11-02-2022,Air India,531,20:00,Delhi,1485,1-stop,20:45,Mumbai,42220,business
3,11-02-2022,Air India,839,21:25,Delhi,1590,1-stop,23:55,Mumbai,44450,business
4,11-02-2022,Air India,544,17:15,Delhi,400,1-stop,23:55,Mumbai,46690,business
5,11-02-2022,Vistara,985,19:50,Delhi,130,non-stop,22:00,Mumbai,50264,business
6,11-02-2022,Air India,479,21:15,Delhi,1065,1-stop,15:00,Mumbai,50669,business
7,11-02-2022,Air India,473,18:40,Delhi,1365,1-stop,17:25,Mumbai,51059,business
8,11-02-2022,Vistara,871,20:35,Delhi,1075,1-stop,14:30,Mumbai,51731,business
9,11-02-2022,Vistara,977,19:00,Delhi,135,non-stop,21:15,Mumbai,53288,business


## There are too many distinct values for departure time, so I'll categorize them into 5 categories that represent the time of day for the flights:

In [44]:
df['dep_time'].value_counts()

07:00    8760
10:25    7693
19:00    5738
09:45    5468
08:30    4955
         ... 
03:30      18
22:05      18
00:15      15
00:45      12
04:15       5
Name: dep_time, Length: 251, dtype: int64

In [45]:
df['dept_time'] = pd.cut(pd.to_datetime(df['dep_time'], format='%H:%M').dt.hour,
                           bins=[0, 4, 8, 12, 17, 24],
                           labels=['late night', 'early morning', 'morning', 'afternoon', 'night'],
                          ordered=False)

In [46]:
df['dept_time'].value_counts()

night            82615
early morning    76419
morning          72054
afternoon        65557
late night        3307
Name: dept_time, dtype: int64

In [47]:
df.isnull().sum()

date                 0
airline              0
num_code             0
dep_time             0
from                 0
time_taken_mins      0
stop                 0
arr_time             0
to                   0
price                0
class                0
dept_time          303
dtype: int64

## I believe some of the times in the dataframe didn't convert to the categories correctly. 

## For sake of time and the fact it's only a very small percentage of the total rows, I will remove them:

In [48]:
df.dropna(subset=['dept_time'], inplace=True)

df.isnull().sum()

date               0
airline            0
num_code           0
dep_time           0
from               0
time_taken_mins    0
stop               0
arr_time           0
to                 0
price              0
class              0
dept_time          0
dtype: int64

## Won't be needing dep_time:

In [49]:
df = df.drop('dep_time', axis=1)
df.head()

,date,airline,num_code,from,time_taken_mins,stop,arr_time,to,price,class,dept_time
0,11-02-2022,Air India,868,Delhi,120,non-stop,20:00,Mumbai,25612,business,night
1,11-02-2022,Air India,624,Delhi,135,non-stop,21:15,Mumbai,25612,business,night
2,11-02-2022,Air India,531,Delhi,1485,1-stop,20:45,Mumbai,42220,business,night
3,11-02-2022,Air India,839,Delhi,1590,1-stop,23:55,Mumbai,44450,business,night
4,11-02-2022,Air India,544,Delhi,400,1-stop,23:55,Mumbai,46690,business,afternoon


## Now that I've cleaned the data, I'm ready to export it as a CSV so I can perform visualizations in Power BI. 

## I'll save the CSV to my desktop:

In [50]:
# import os

In [51]:
# filename = 'cleaned_flights_2.csv'

# desktop_path = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
# file_path = os.path.join(desktop_path, filename)

# df.to_csv(file_path, index=False)

# print(f'{filename} exported successfully to desktop!')

## Before I do some price prediction and fit a model, I need to create dummy variables for anything categorical that I want to use:

In [52]:
df['date'].value_counts()

07-03-2022    6626
28-02-2022    6597
21-03-2022    6587
14-03-2022    6579
08-03-2022    6570
06-03-2022    6536
01-03-2022    6534
15-03-2022    6529
13-03-2022    6528
22-03-2022    6528
10-03-2022    6520
23-03-2022    6519
20-03-2022    6506
12-03-2022    6498
24-03-2022    6496
02-03-2022    6495
04-03-2022    6488
18-03-2022    6484
03-03-2022    6476
19-03-2022    6473
25-03-2022    6466
26-03-2022    6433
21-02-2022    6415
27-02-2022    6413
16-03-2022    6405
05-03-2022    6398
23-02-2022    6397
11-03-2022    6391
22-02-2022    6378
09-03-2022    6354
24-02-2022    6346
25-02-2022    6335
27-03-2022    6305
17-03-2022    6289
26-02-2022    6269
28-03-2022    6148
31-03-2022    6145
30-03-2022    6066
29-03-2022    6060
20-02-2022    5820
18-02-2022    5765
16-02-2022    5738
17-02-2022    5700
19-02-2022    5666
15-02-2022    5395
14-02-2022    5078
13-02-2022    4250
12-02-2022    4031
11-02-2022    1927
Name: date, dtype: int64

## I notice there are only flights for months February and March. I'll create 2 new binary columns that tell me if a flight is in that month:

In [53]:
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')

df['in_February'] = (df['date'].dt.month == 2).astype(int)
df['in_March'] = (df['date'].dt.month == 3).astype(int)

## Now I don't need the date column:

In [54]:
df = df.drop('date', axis=1)

df.head()

,airline,num_code,from,time_taken_mins,stop,arr_time,to,price,class,dept_time,in_February,in_March
0,Air India,868,Delhi,120,non-stop,20:00,Mumbai,25612,business,night,1,0
1,Air India,624,Delhi,135,non-stop,21:15,Mumbai,25612,business,night,1,0
2,Air India,531,Delhi,1485,1-stop,20:45,Mumbai,42220,business,night,1,0
3,Air India,839,Delhi,1590,1-stop,23:55,Mumbai,44450,business,night,1,0
4,Air India,544,Delhi,400,1-stop,23:55,Mumbai,46690,business,afternoon,1,0


## I don't need arrival time nor num_code. Let's drop them:

In [55]:
df = df.drop('arr_time', axis=1)
df = df.drop('num_code', axis=1)

df.head()

,airline,from,time_taken_mins,stop,to,price,class,dept_time,in_February,in_March
0,Air India,Delhi,120,non-stop,Mumbai,25612,business,night,1,0
1,Air India,Delhi,135,non-stop,Mumbai,25612,business,night,1,0
2,Air India,Delhi,1485,1-stop,Mumbai,42220,business,night,1,0
3,Air India,Delhi,1590,1-stop,Mumbai,44450,business,night,1,0
4,Air India,Delhi,400,1-stop,Mumbai,46690,business,afternoon,1,0


In [56]:
df['to'].value_counts()

Mumbai       59043
Delhi        57198
Bangalore    51112
Kolkata      49456
Hyderabad    42775
Chennai      40368
Name: to, dtype: int64

In [57]:
df['from'].value_counts()

Delhi        61328
Mumbai       60758
Bangalore    51960
Kolkata      46347
Hyderabad    40859
Chennai      38700
Name: from, dtype: int64

In [58]:
df['airline'].value_counts()

Vistara      127859
Air India     80892
Indigo        42960
GO FIRST      23173
AirAsia       15955
SpiceJet       9011
StarAir          61
Trujet           41
Name: airline, dtype: int64

In [59]:
df['class'].value_counts()

economy     206465
business     93487
Name: class, dtype: int64

In [60]:
df['stop'].value_counts()

1-stop      250696
non-stop     35970
2-stops      13286
Name: stop, dtype: int64

In [61]:
df['dept_time'].value_counts()

night            82615
early morning    76419
morning          72054
afternoon        65557
late night        3307
Name: dept_time, dtype: int64

## Let's actually make those dummies now:

In [62]:
airline_dummies = pd.get_dummies(df['airline'], prefix='airline')
from_dummies = pd.get_dummies(df['from'], prefix='from')
stop_dummies = pd.get_dummies(df['stop'], prefix='stop')
to_dummies = pd.get_dummies(df['to'], prefix='to')
class_dummies = pd.get_dummies(df['class'], prefix='class')
dept_time_dummies = pd.get_dummies(df['dept_time'], prefix='dept_time')

df = pd.concat([df, airline_dummies, from_dummies, stop_dummies, to_dummies, class_dummies, dept_time_dummies], axis=1)

df.drop(['airline', 'from', 'stop', 'to', 'class', 'dept_time'], axis=1, inplace=True)

In [63]:
df.head()

,time_taken_mins,price,in_February,in_March,airline_Air India,airline_AirAsia,airline_GO FIRST,airline_Indigo,airline_SpiceJet,airline_StarAir,...,to_Hyderabad,to_Kolkata,to_Mumbai,class_business,class_economy,dept_time_late night,dept_time_early morning,dept_time_morning,dept_time_afternoon,dept_time_night
0,120,25612,1,0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
1,135,25612,1,0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
2,1485,42220,1,0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
3,1590,44450,1,0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
4,400,46690,1,0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0


In [64]:
df.describe()

,time_taken_mins,price,in_February,in_March,airline_Air India,airline_AirAsia,airline_GO FIRST,airline_Indigo,airline_SpiceJet,airline_StarAir,...,to_Hyderabad,to_Kolkata,to_Mumbai,class_business,class_economy,dept_time_late night,dept_time_early morning,dept_time_morning,dept_time_afternoon,dept_time_night
count,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,...,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000
mean,733.422464,20901.148837,0.335120,0.664880,0.269683,0.053192,0.077256,0.143223,0.030041,0.000203,...,0.142606,0.164880,0.196841,0.311673,0.688327,0.011025,0.254771,0.240218,0.218558,0.275427
std,431.621884,22701.023616,0.472033,0.472033,0.443796,0.224416,0.266997,0.350301,0.170702,0.014259,...,0.349671,0.371073,0.397612,0.463178,0.463178,0.104420,0.435733,0.427217,0.413269,0.446730
min,50.000000,1105.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,410.000000,4784.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,675.000000,7425.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,970.000000,42521.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,2990.000000,123071.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Now that we have all of our dummy variables, let's start checking correlations:

In [65]:
corr = df.corr()['price']
corr.sort_values(ascending=False)

price                      1.000000
class_business             0.937842
airline_Vistara            0.360539
time_taken_mins            0.204003
stop_1-stop                0.200066
in_February                0.092159
airline_Air India          0.069756
dept_time_night            0.034014
to_Kolkata                 0.021256
from_Chennai               0.018551
to_Chennai                 0.018278
from_Kolkata               0.015914
from_Mumbai                0.013829
to_Bangalore               0.013515
dept_time_morning          0.013413
from_Bangalore             0.012163
to_Mumbai                  0.010640
dept_time_early morning   -0.001476
to_Hyderabad              -0.008815
airline_Trujet            -0.009094
airline_StarAir           -0.010032
from_Hyderabad            -0.013427
dept_time_afternoon       -0.034346
from_Delhi                -0.043457
to_Delhi                  -0.051815
dept_time_late night      -0.058304
stop_2-stops              -0.064370
in_March                  -0

## Let's fit the first model with all of the predictors first: 

## Model 1

In [66]:
y = df["price"]
X = df.drop("price", axis=1)

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model1 = LinearRegression().fit(X_train, y_train)

print("Training set R-squared score:", model1.score(X_train, y_train))

print("Test set R-squared score:", model1.score(X_test, y_test))

Training set R-squared score: 0.9112189200080091
Test set R-squared score: 0.909360529476755


In [68]:
train_score = model1.score(X_train, y_train)
test_score = model1.score(X_test, y_test)

In [69]:
score_diff = train_score - test_score

if score_diff > 0.1:
    print("The model is overfitting.")
elif 0 <= score_diff <= 0.1:
    print("The model may be slightly overfitting.")
else:
    print("The model is not overfitting.")

The model may be slightly overfitting.


In [70]:
train_score - test_score

0.0018583905312540283

## I can see that I'm having trouble with some overfit. This isn't surprising given that I used all columns as predictors. Let's try and remove some of the columns with the lowest correlations to price:

## For this model, we'll keep only those columns with an absolute correlation to price above .1:

## Model 2

In [71]:
X1 = X[['class_business', 'airline_Vistara', 'time_taken_mins', 'stop_1-stop', 'airline_SpiceJet', 'airline_AirAsia', 'stop_non-stop', 'airline_GO FIRST', 'airline_Indigo', 'class_economy']]

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=42)

model2 = LinearRegression().fit(X_train, y_train)

print("Training set R-squared score:", model2.score(X_train, y_train))

print("Test set R-squared score:", model2.score(X_test, y_test))

Training set R-squared score: 0.901593141439151
Test set R-squared score: 0.9002629316225305


In [73]:
train_score = model2.score(X_train, y_train)
test_score = model2.score(X_test, y_test)

In [74]:
score_diff = train_score - test_score

if score_diff > 0.1:
    print("The model is overfitting.")
elif 0 <= score_diff <= 0.1:
    print("The model may be slightly overfitting.")
else:
    print("The model is not overfitting.")

The model may be slightly overfitting.


In [75]:
train_score - test_score

0.0013302098166205756

## After removing a lot of columns, we still have strong accuracy in the model, but I'm still seeing the slightest bit of overfit - which is not bad - but I'd like to see if I can completely eliminate it first before moving on.

## Let's try a new model with predictors having an absolute correlation higher than .2:

## Model 3

In [76]:
X2 = X[['class_business', 'airline_Vistara', 'time_taken_mins', 'stop_1-stop', 'airline_Indigo', 'class_economy']]

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=42)

model3 = LinearRegression().fit(X_train, y_train)

print("Training set R-squared score:", model3.score(X_train, y_train))

print("Test set R-squared score:", model3.score(X_test, y_test))

Training set R-squared score: 0.8954296553487908
Test set R-squared score: 0.8942163514440682


In [78]:
train_score = model3.score(X_train, y_train)
test_score = model3.score(X_test, y_test)

In [79]:
score_diff = train_score - test_score

if score_diff > 0.1:
    print("The model is overfitting.")
elif 0 <= score_diff <= 0.1:
    print("The model may be slightly overfitting.")
else:
    print("The model is not overfitting.")

The model may be slightly overfitting.


In [80]:
train_score - test_score

0.001213303904722629

## I'm going to try going even simpler:

## Model 4

In [81]:
corr = X.corr()['class_business']
corr.sort_values(ascending=False)

class_business             1.000000
airline_Vistara            0.301848
time_taken_mins            0.138489
airline_Air India          0.124660
stop_1-stop                0.119814
dept_time_night            0.044295
from_Mumbai                0.016501
in_February                0.013642
to_Chennai                 0.012156
to_Mumbai                  0.010660
from_Chennai               0.010033
to_Hyderabad               0.007064
to_Bangalore               0.005126
from_Bangalore             0.004688
from_Hyderabad             0.002840
dept_time_early morning    0.000913
dept_time_morning         -0.002987
airline_Trujet            -0.007868
airline_StarAir           -0.009597
to_Kolkata                -0.012727
in_March                  -0.013642
from_Delhi                -0.014247
from_Kolkata              -0.019359
to_Delhi                  -0.020522
dept_time_afternoon       -0.032280
dept_time_late night      -0.053332
stop_non-stop             -0.068881
stop_2-stops              -0

### 'class_business' and 'class_economy' have perfect multicollinearity at -1.0, so I'll remove one:

In [82]:
X3 = X[['class_business', 'time_taken_mins']]

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.2, random_state=42)

model4 = LinearRegression().fit(X_train, y_train)

print("Training set R-squared score:", model4.score(X_train, y_train))

print("Test set R-squared score:", model4.score(X_test, y_test))

Training set R-squared score: 0.885551802538876
Test set R-squared score: 0.8835191636894886


In [84]:
train_score = model4.score(X_train, y_train)
test_score = model4.score(X_test, y_test)

In [85]:
score_diff = train_score - test_score

if score_diff > 0.1:
    print("The model is overfitting.")
elif 0 <= score_diff <= 0.1:
    print("The model may be slightly overfitting.")
else:
    print("The model is not overfitting.")

The model may be slightly overfitting.


In [86]:
train_score - test_score

0.002032638849387425

## Another try:

## Model 5

In [87]:
X4 = X[['class_business', 'class_economy', 'stop_non-stop', 'time_taken_mins', 'stop_1-stop', 'stop_2-stops']]

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X4, y, test_size=0.2, random_state=42)

model5 = LinearRegression().fit(X_train, y_train)

print("Training set R-squared score:", model5.score(X_train, y_train))

print("Test set R-squared score:", model5.score(X_test, y_test))

Training set R-squared score: 0.8957947249400068
Test set R-squared score: 0.8942399918707101


In [89]:
train_score = model5.score(X_train, y_train)
test_score = model5.score(X_test, y_test)

In [90]:
score_diff = train_score - test_score

if score_diff > 0.1:
    print("The model is overfitting.")
elif 0 <= score_diff <= 0.1:
    print("The model may be slightly overfitting.")
else:
    print("The model is not overfitting.")

The model may be slightly overfitting.


In [91]:
train_score - test_score

0.0015547330692966543

## Last one:

## Model 6

In [92]:
X5 = X[['class_business', 'class_economy']]

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X5, y, test_size=0.2, random_state=42)

model6 = LinearRegression().fit(X_train, y_train)

print("Training set R-squared score:", model6.score(X_train, y_train))

print("Test set R-squared score:", model6.score(X_test, y_test))

Training set R-squared score: 0.8800577915985761
Test set R-squared score: 0.8774878060874978


In [94]:
train_score = model6.score(X_train, y_train)
test_score = model6.score(X_test, y_test)

In [95]:
score_diff = train_score - test_score

if score_diff > 0.1:
    print("The model is overfitting.")
elif 0 <= score_diff <= 0.1:
    print("The model may be slightly overfitting.")
else:
    print("The model is not overfitting.")

The model may be slightly overfitting.


In [96]:
train_score - test_score

0.002569985511078343

## After evaluating all the models, I'm going to choose Model 4. This model only requires 2 predictors and still has high accuracy with very little change in overfit.

In [97]:
df.describe()

,time_taken_mins,price,in_February,in_March,airline_Air India,airline_AirAsia,airline_GO FIRST,airline_Indigo,airline_SpiceJet,airline_StarAir,...,to_Hyderabad,to_Kolkata,to_Mumbai,class_business,class_economy,dept_time_late night,dept_time_early morning,dept_time_morning,dept_time_afternoon,dept_time_night
count,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,...,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000,299952.000000
mean,733.422464,20901.148837,0.335120,0.664880,0.269683,0.053192,0.077256,0.143223,0.030041,0.000203,...,0.142606,0.164880,0.196841,0.311673,0.688327,0.011025,0.254771,0.240218,0.218558,0.275427
std,431.621884,22701.023616,0.472033,0.472033,0.443796,0.224416,0.266997,0.350301,0.170702,0.014259,...,0.349671,0.371073,0.397612,0.463178,0.463178,0.104420,0.435733,0.427217,0.413269,0.446730
min,50.000000,1105.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,410.000000,4784.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,675.000000,7425.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,970.000000,42521.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,2990.000000,123071.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [98]:
# X3 = X[['class_business', 'time_taken_mins']]

In [99]:
new_flight_one = np.array([[1.0, 675]])

predicted_price1 = model4.predict(new_flight_one)

predicted_price1

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([51984.86612551])

In [100]:
new_flight_two = np.array([[0.0, 675]])

predicted_price2 = model4.predict(new_flight_two)

predicted_price2

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([6502.16851736])

## We can see from this prediction that flights of equal length create varying prices based off class. The economy class is clearly a lot cheaper.

## This model can be a good way to know how to price tickets based on the class of ticket and length of a flight.

## Let's test more flights to confirm:

In [101]:
new_flight_three = np.array([[1.0, 1500]])

predicted_price3 = model4.predict(new_flight_three)

predicted_price3

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([55235.64281478])

In [102]:
new_flight_four = np.array([[0, 1500]])

predicted_price4 = model4.predict(new_flight_four)

predicted_price4

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([9752.94520663])

## For these flights, we increased the time_taken_mins to reflect increases in both classes of ticket while keeping a similar price ratio from business to economy.

# Overall in this notebook:

### - Cleaned dataset: Removed duplicates + null values and fixed errors in the 'stop' column values
### - Performed feature engineering to convert departure time into 5 categories representing the time of day
### - Exported the cleaned data to a CSV file in order to create visualizations in PowerBI
### - Created dummy variables in order to fit a linear regression model
### - Fit a Linear Regression model using 2 predictors of price
### - Used the model to predict the price of flights based off class and total time taken in minutes.